In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import scipy.stats as sp
import scipy.optimize as sco
import plotly
from plotly import graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected = True)

In [2]:
def BSM(S0, K, tau, r, sigma, opt_type='c', q=0):
    d1 = (np.log(S0/K)+(r-q+.5*sigma**2)*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)
    N = lambda x: sp.norm.cdf(x)
    if opt_type == 'c':
        return S0*np.exp(-q*tau)*N(d1) - np.exp(-r*tau)*K*N(d2)
    else:
        return K*np.exp(-r*tau)*N(-d2) - S0*np.exp(-q*tau)*N(-d1)

In [3]:
files = ['20180427', '20180504', '20180511', '20180518', '20180525', '20180601', '20180608', '20180615',
        '20180720', '20180817', '20180921', '20181019', '20181116', '20190118', '20190215', '20190621',
        '20200117']
r = 0.0184 # 3m Treasury 4/30/18
dfs = []
for f in files:
    temp = pd.read_csv('{}.csv'.format(f))
    temp['TTM'] = (pd.to_datetime(temp['T'].apply(lambda x: str(x))) - pd.datetime(2018,4,27)).apply(lambda x: x.days)/365
    dfs.append(temp)
S = dfs[0].S[0]
dfs[0].TTM = 1/365
dfs

[     Unnamed: 0         T       K   P_bid   P_ask   C_bid   C_ask        S  \
 0             0  20180427   910.0    0.00    0.02  607.10  611.80  1517.96   
 1             1  20180427   920.0    0.00    0.03  596.50  601.50  1517.96   
 2             2  20180427   930.0    0.00    3.80  586.70  591.70  1517.96   
 3             3  20180427   940.0    0.00    0.05  576.50  581.50  1517.96   
 4             4  20180427   950.0    0.00    0.05  566.50  571.50  1517.96   
 5             5  20180427   960.0    0.00    1.67  556.40  561.40  1517.96   
 6             6  20180427   970.0    0.00    3.35  546.50  551.50  1517.96   
 7             7  20180427   980.0    0.00    4.25  536.50  541.50  1517.96   
 8             8  20180427   990.0    0.00    0.68  526.55  531.55  1517.96   
 9             9  20180427  1000.0    0.00    0.04  516.55  521.55  1517.96   
 10           10  20180427  1010.0    0.00    0.68  506.55  511.55  1517.96   
 11           11  20180427  1020.0    0.00    0.08  

In [4]:
tol = 10E-9
def SVI(df,a,b,p,m,sig):
    k = np.log(np.asarray(df.K)/(1517.96*np.exp(r*np.asarray(df.TTM))))
    return a + b*(p*(np.asarray(k)-m) + np.sqrt((np.asarray(k)-m)**2 + sig**2))
def SVI_fit(df,IV_col):
    obj = lambda x: np.sum((SVI(df, x[0], x[1], x[2], x[3], x[4]) - np.asarray(df[IV_col]))**2)
    cons = ({'type':'ineq','fun': lambda x: x[0] + x[1]*x[4]*np.sqrt(1 - x[2]**2)},
            {'type':'ineq','fun': lambda x: x[1]},
            {'type':'ineq','fun': lambda x: x[4] - tol})
    bnds = ((None,None),(0,None),(-1+tol,1-tol),(None,None),(tol,None))
    guess = [.01,.01,.01,.01,.01]
    result = sco.minimize(obj,guess,constraints=cons,bounds=bnds,tol=tol)
    return result

SVI_fit(dfs[0],'Comb_vol')

     fun: 1.6554086364248797
     jac: array([-0.00129673, -0.0003247 , -0.00082405,  0.00422643, -0.01618318])
 message: 'Optimization terminated successfully.'
    nfev: 392
     nit: 52
    njev: 52
  status: 0
 success: True
       x: array([-2.19707057, 11.7338407 ,  0.11266391,  0.04539798,  0.28811341])

In [6]:
temp = dfs[0]
SVI_fit(temp[temp.K < S],'Comb_vol')

     fun: 0.20880556411566992
     jac: array([-1.50144100e-04, -3.88436019e-05,  8.08741897e-05, -5.24593517e-04,
        2.27026641e-04])
 message: 'Optimization terminated successfully.'
    nfev: 341
     nit: 46
    njev: 46
  status: 0
 success: True
       x: array([ 1.12229365,  4.79704925, -0.61302288, -0.09906122,  0.03579519])

In [7]:
SVI_fit(temp[temp.K > S],'Comb_vol')

     fun: 0.061591317214405125
     jac: array([-0.06196412, -0.00759571, -0.0278478 ,  0.24291823, -0.28382206])
 message: 'Iteration limit exceeded'
    nfev: 744
     nit: 101
    njev: 101
  status: 9
 success: False
       x: array([0.87241389, 6.28787734, 0.22845659, 0.07466522, 0.04658161])

In [14]:
fit1 = SVI_fit(temp[temp.K < S],'Comb_vol')['x']
fit2 = SVI_fit(temp[temp.K > S],'Comb_vol')['x']
df427 = pd.DataFrame({'K': np.linspace(800,2200,2801), 'TTM': dfs[0].TTM[0]})
df427['Sigma'] = np.append(SVI(df427[df427.K < S],fit1[0],fit1[1],fit1[2],fit1[3],fit1[4]),
          SVI(df427[df427.K > S],fit2[0],fit2[1],fit2[2],fit2[3],fit2[4]), axis=0)
df427['C'] = df427.apply(lambda x: BSM(S,x.K,x.TTM,r,x.Sigma), axis=1)
df427

,K,TTM,Sigma,C
0,800.0,0.00274,5.317955,719.110577
1,800.5,0.00274,5.313127,718.610463
2,801.0,0.00274,5.308302,718.110348
3,801.5,0.00274,5.303480,717.610234
4,802.0,0.00274,5.298661,717.110120
5,802.5,0.00274,5.293845,716.610006
6,803.0,0.00274,5.289032,716.109892
7,803.5,0.00274,5.284222,715.609778
8,804.0,0.00274,5.279415,715.109665
9,804.5,0.00274,5.274611,714.609551


In [24]:
trace = go.Scatter(x = df427['K'], 
                   y = (df427.C.shift(-1) - 2*df427.C + df427.C.shift(1))/.5**2, name = 'f(S)')
data = [trace]
layout = go.Layout(title = 'AMZN PDF: 4-27-2018', yaxis = dict(title = 'Probability Density'), xaxis = dict(title = 'S'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [23]:
temp1 = df427[(df427.K < 1517.5)]
temp2 = df427[(df427.K > 1518)]
trace = go.Scatter(x = temp1['K'], 
                   y = (temp1.C.shift(-1) - 2*temp1.C + temp1.C.shift(1))/.5**2, name = 'f(S)')
trace0 = go.Scatter(x = temp2['K'], 
                   y = (temp2.C.shift(-1) - 2*temp2.C + temp2.C.shift(1))/.5**2, name = 'f(S)')
trace1 = go.Scatter(x = 100*[S], y = np.linspace(0,0.005,100), name = 'Underlying')
data = [trace,trace0,trace1]
layout = go.Layout(title = 'AMZN PDF: 4-27-2018', yaxis = dict(title = 'Probability Density'), xaxis = dict(title = 'S'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [16]:
trace0 = go.Scatter(
    x = dfs[0].K, y = 100*dfs[0].Comb_vol, name = 'Implied Volatility'
)
trace1 = go.Scatter(
    x = df427.K, y = 100*df427.Sigma, name = 'SVI Fit'
)
trace2 = go.Scatter(
    x = df427.K, y = (df427.C.shift(-1) - 2*df427.C + df427.C.shift(1))/.5**2, name = 'PDF', yaxis = 'y2'
)

data = [trace0,trace1]
layout = go.Layout(
    title = 'AMZN Implied Volatility: 4-27-2018', 
    yaxis = dict(title = 'Implied Vol (%)'), 
    yaxis2 = dict(title = 'Probability Density', side = 'right'), xaxis = dict(title = 'K'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [17]:
S

1517.96